# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234079180039                   -0.50    7.0         
  2   -7.250205314131       -1.79       -1.40    1.0   8.09ms
  3   -7.251164179010       -3.02       -2.20    1.0   8.00ms
  4   -7.251279655437       -3.94       -2.22    2.0   9.70ms
  5   -7.251333123833       -4.27       -2.68    1.0   7.97ms
  6   -7.251338230701       -5.29       -3.24    1.0   8.01ms
  7   -7.251338748521       -6.29       -3.64    2.0   9.60ms
  8   -7.251338789414       -7.39       -4.02    1.0   8.20ms
  9   -7.251338797339       -8.10       -4.52    1.0   8.25ms
 10   -7.251338798612       -8.90       -5.03    2.0   9.89ms
 11   -7.251338798695      -10.08       -5.73    1.0   8.97ms
 12   -7.251338798702      -11.18       -5.80    3.0   12.1ms
 13   -7.251338798704      -11.60       -6.22    1.0   8.81ms
 14   -7.251338798704      -12.42       -6.63    2.0   10.3ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.043612459990155304
[ Info: Arnoldi iteration step 2: normres = 0.38688212677679984
[ Info: Arnoldi iteration step 3: normres = 0.7006995498786475
[ Info: Arnoldi iteration step 4: normres = 0.5138614746166312
[ Info: Arnoldi iteration step 5: normres = 0.18821166807851542
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.29e-02, 1.81e-02, 1.52e-01, 9.54e-02, 3.04e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5820801896766755
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.41e-02, 1.23e-02, 5.37e-01, 1.92e-01, 6.90e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10334869765680543
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.07e-03, 5.53e-03, 7.71e-02, 3.24e-02, 5.02e-02)
[ Info: Arnoldi iteration step 8: normres = 0.08763118398210602
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.99